I used this project to play about with Gradio blocks a little bit as it had more inputs than the other projects I've done.
Its a password generator which I have no doubt I will use!

In [ ]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import requests
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr

In [ ]:
# keys

load_dotenv(override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")

if openai_api_key:
    print("All good")
else:
    print("OpenAI key issue")

claude_api_key = os.getenv("ANTHROPIC_API_KEY")

if claude_api_key:
    print("All good")
else:
    print("Claude key issue")

google_api_key = os.getenv("GOOGLE_API_KEY")

if google_api_key:
    print("All good")
else:
    print("Google key issue")


In [ ]:
# initialise

openai = OpenAI()
claude = anthropic.Anthropic()
google.generativeai.configure()

OPENAI_MODEL = "gpt-4o"
CLAUDE_MODEL = "claude-sonnet-4-20250514"
GOOGLE_MODEL = "gemini-2.0-flash"

max_tok = 500

In [ ]:
#Prompts

def pass_system_prompt(required_len, spec_char="Y",num_char="Y",min_lowercase=1,min_uppercase=1):

    system_prompt = f"""You are a secure password generator. Your task is to create a single, cryptographically strong password that meets ALL specified requirements.
    
CRITICAL REQUIREMENTS:
- Length: EXACTLY {required_len} characters
- Must include: At least {min_lowercase} lowercase letter(s) AND at least {min_uppercase} uppercase letter(s)
- Special characters: {'REQUIRED - include at least 1 char' if spec_char else 'FORBIDDEN - do not include any'}
- Numbers: {'REQUIRED - include at least 1 digit' if num_char else 'FORBIDDEN - do not include any digits'}

SECURITY RULES:
1. Generate truly random passwords - avoid patterns, dictionary words, or predictable sequences
2. Distribute character types evenly throughout the password
3. Do not use repeated characters excessively (max 2 of same character)
4. Ensure password meets minimum complexity for each required character type

OUTPUT FORMAT:
- Respond with ONLY the generated password
- No explanations, no additional text, just the password
- Verify the password meets ALL requirements before responding"""

    return system_prompt

def pass_user_prompt(required_len, spec_char="Y",num_char="Y",min_lowercase=1,min_uppercase=1):
    
    user_prompt = f"""Generate a secure password with these exact specifications:
    
Length: {required_len} characters
Lowercase letters: Required (minimum {min_lowercase})
Uppercase letters: Required (minimum {min_uppercase})
Numbers: {'Required (minimum 1)' if num_char else 'Not allowed'}
Special characters: {'Required (minimum 1)' if spec_char else 'Not allowed'}

Requirements verification checklist:
✓ Exactly {required_len} characters total
✓ Contains {min_lowercase}+ lowercase letters
✓ Contains {min_uppercase}+ uppercase letters
✓ {'Contains 1+ numbers' if num_char else 'Contains NO numbers'}
✓ {'Contains 1+ special characters' if spec_char else 'Contains NO special characters'}
✓ No obvious patterns or dictionary words
✓ Good distribution of character types

Generate the password now."""

    return user_prompt
        
def pass_messages(required_len, spec_char,num_char,min_lowercase,min_uppercase):
    messages = [
        {"role":"system","content":pass_system_prompt(required_len, spec_char,num_char,min_lowercase,min_uppercase)},
        {"role":"user","content":pass_user_prompt(required_len, spec_char,num_char,min_lowercase,min_uppercase)}
    ]

    return messages


In [ ]:
#test

messages1 = pass_messages(12, "N", "Y",1,1)
print(messages1)


In [ ]:
def openai_password_gen(required_len, spec_char, num_char,min_lowercase,min_uppercase):
    response=openai.chat.completions.create(
        model=OPENAI_MODEL,
        max_tokens=max_tok,
        messages=pass_messages(required_len, spec_char,num_char,min_lowercase,min_uppercase)
    )
    return response.choices[0].message.content
                                           

In [ ]:
def claude_password_gen(required_len, spec_char, num_char,min_lowercase,min_uppercase):
    response = claude.messages.create(
        model=CLAUDE_MODEL,
        max_tokens=max_tok,
        system=pass_system_prompt(required_len, spec_char, num_char,min_lowercase,min_uppercase),
        messages = [{"role":"user","content":pass_user_prompt(required_len, spec_char, num_char,min_lowercase,min_uppercase)}]
    )
    return response.content[0].text
        

In [ ]:
def google_password_gen(required_len, spec_char, num_char,min_lowercase,min_uppercase):
    message = google.generativeai.GenerativeModel(
        model_name=GOOGLE_MODEL,
        system_instruction=pass_system_prompt(required_len, spec_char, num_char,min_lowercase,min_uppercase)
    )
    response = message.generate_content(pass_user_prompt(required_len, spec_char, num_char,min_lowercase,min_uppercase))
    return response.text

In [ ]:
#test
messages1 = openai_password_gen(12, "N","Y",1,1)
messages2 = claude_password_gen(12,"N","Y",1,1)
messages3= google_password_gen(12,"N","Y",1,1)
print("OpenAI: ",messages1)
print("Claude: ", messages2)
print("Gemini: ", messages3)

In [ ]:
# model select

def select_model(required_len, spec_char, num_char,min_lowercase,min_uppercase,model):
    if model == "OpenAI":
        return openai_password_gen(required_len, spec_char, num_char,min_lowercase,min_uppercase)
    elif model == "Claude":
        return claude_password_gen(required_len, spec_char, num_char,min_lowercase,min_uppercase)
    elif model == "Gemini":
        return google_password_gen(required_len, spec_char, num_char,min_lowercase,min_uppercase)
    else:
        print("No model selected")
        return None

In [ ]:
test = select_model(12, "N","Y",1,1,"OpenAI")

print(test)

In [ ]:
css = """
#password_box textarea {
    background-color: #306998;
    color: white;
}
"""

In [ ]:

with gr.Blocks(css=css) as demo:
    gr.Markdown("Choose your password complexity requirements and run:")
    with gr.Row():
        with gr.Column(min_width=150,scale=2):
            with gr.Row():
                required_len = gr.Number(label="Specify the required length",value=12,minimum=1,maximum=30)
                min_lowercase = gr.Number(label="the minimum lowercase letters", value=1,minimum=0)
                min_uppercase = gr.Number(label="the minimum uppercase letters", value=1,minimum=0)
        with gr.Column():
            spec_char = gr.Checkbox(label="Include special characters?",value=True)
            num_char = gr.Checkbox(label="Include numbers?", value=True)
    with gr.Row():
        with gr.Column():
            model = gr.Dropdown(["OpenAI","Claude","Gemini"])
            btn = gr.Button("Run")
        with gr.Column():
            output = gr.Textbox(label="Password:", elem_id="password_box")
    
    btn.click(fn=select_model,inputs=[required_len,spec_char,num_char,min_lowercase,min_uppercase,model],outputs=output)

demo.launch()
    